# Overview

Here's a simple implementation of a Variational Autoencoder (VAE) using Python with TensorFlow and Keras. This example will demonstrate how to build a VAE to generate images from the MNIST dataset, which consists of handwritten digits.

## Prerequisites

Make sure you have the folliwing libraries installed:

In [ ]:
pip install tensorflow numpy matplotlib # type: ignore

## Code Implementtion

Here's the complete code for a simple VAE:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Define the VAE model
latent_dim = 2  # Dimensionality of the latent space

# Encoder
encoder_inputs = layers.Input(shape=(28, 28, 1))
x = layers.Flatten()(encoder_inputs)
x = layers.Dense(128, activation='relu')(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(128, activation='relu')(decoder_inputs)
x = layers.Dense(28 * 28, activation='sigmoid')(x)
decoder_outputs = layers.Reshape((28, 28, 1))(x)

# Build the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
decoder = models.Model(decoder_inputs, decoder_outputs, name="decoder")

# VAE model
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, vae_outputs, name="vae")

# Loss function
def vae_loss(original, reconstructed):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(original, reconstructed)
    reconstruction_loss *= 28 * 28  # Scale by the number of pixels
    kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
    return tf.reduce_mean(reconstruction_loss + kl_loss)

# Compile the model
vae.compile(optimizer='adam', loss=vae_loss)

# Train the VAE
vae.fit(x_train, x_train, epochs=30, batch_size=128, validation_data=(x_test, x_test))

# Generate new images from the VAE
def generate_images(num_images):
    z_samples = np.random.normal(size=(num_images, latent_dim))
    generated_images = decoder.predict(z_samples)
    return generated_images

# Display generated images
num_images = 10
generated_images = generate_images(num_images)

plt.figure(figsize=(10, 2))
for i in range(num_images):
    ax = plt.subplot(2, 10, i + 1)
    plt.imshow(generated_images[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

plt.show()
